# ML Parsing #
Initial parsing of the grade submissions. It is meant only for research purposes and not for production environments.

_What it does?_
- Parses the intial csv files (form csv + grades uploaded in csv format)
- Shows basic info of the data
- Connects form submissions with their associated grades and cleans entry with no grades
- Parses the submission for the CSD Department
- Drops invalid values and fixes any other issues with the dataset
- Produces a final dataset with the form submissions and the grades. Each course found in the dataset is represented as a column. The initial dataset produced 111 columns/unique courses*. 

*Some columns refer to the same course but they are with different codes (old/new study program)

**ATTENTION!!: Some outputs have been deleted for privacy reasons**

In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## General Parsing ##

In [ ]:
# reading the form submissions (download csv file)
form_csv = "Grade Collection - Form Responses 1.csv"
submissions_csv = pd.read_csv(form_csv)
display(submissions_csv)

In [4]:
submissions_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 16 columns):
Timestamp                                                                   53 non-null object
Ηλικία                                                                      53 non-null int64
Φύλο                                                                        53 non-null object
Σχολή Φοίτησης                                                              53 non-null object
Επέλεξα τη σχολή μου διότι:                                                 53 non-null object
Κατά μέσο όρο την εβδομάδα, διαβάζω:                                        53 non-null object
Μέσα στο εξάμηνο, παρακαλουθώ:                                              53 non-null object
Υπήρξε ανάγκη για φροντηστηριακή βοήθεια σε κάποιο μάθημα έως τώρα;         53 non-null object
Μετά το πτυχίο, θα ήθελα να ακολουθήσω:                                     53 non-null object
Ποιο από τα παρακάτω ισχύει;               

In [5]:
#Create an extract column to store the paths of the csvs with the grades
csv = [None] * (submissions_csv.shape[0])
submissions_csv["CSV"] = pd.Series(csv).to_numpy()
# display(submissions_csv)
# print(submissions_csv['Filename'])

In [ ]:
# Find associated csv
folder_with_grade_csvs = "GradeParser/csv/"
for r,d,f in os.walk(folder_with_grade_csvs):
    for file in f:
        if ".csv" in file: # For each csv
            without_ext = file.replace("_results.csv","") #Remove extensions added by the parser
            # Set the csv found 
            submissions_csv.loc[submissions_csv["Filename"].str.contains(without_ext,regex=False),"CSV"] = r + file
display(submissions_csv)     

In [ ]:
# Drop those entries with no csv associated 
submissions_csv.dropna(subset=['CSV'],inplace = True)
display(submissions_csv)

## Parsing for Departments ##

In [8]:
csd = submissions_csv[submissions_csv['Σχολή Φοίτησης'] == "Πληροφορική ΑΠΘ"]
hmmy = submissions_csv[submissions_csv['Σχολή Φοίτησης'] == "ΗΜΜΥ ΑΠΘ"]
print("CSD:" + str(csd.shape[0]))
print("HMMY:" + str(hmmy.shape[0]))

CSD:39
HMMY:12


### CSD Department - Grade Statistics ###

In [9]:
# Concat all grades (all the grade csvs together)
csd_csv_files = csd['CSV'].to_numpy()
li = []
# pd.read_csv(csd_csv_files[5],encoding='utf-8')
colnames = ["C_ID","C_NAME","TYPE","EXAM_YEAR","EXAM_PERIOD","ECTS","DM","FACTOR","GRADE"]
# coltypes = {"C_ID":str,"C_NAME":str,"TYPE":str,"EXAM_YEAR":str,"EXAM_PERIOD":str,"ECTS":np.int32,"DM":np.int32,"FACTOR":np.int32,"GRADE":np.float64}
for csv_file in csd_csv_files:
    f = open(csv_file,encoding='utf-8')
    df = pd.read_csv(f, index_col=None, header=None, names = colnames)
    li.append(df)

csd_grades = pd.concat(li, axis=0, ignore_index=True)
display(csd_grades)

,C_ID,C_NAME,TYPE,EXAM_YEAR,EXAM_PERIOD,ECTS,DM,FACTOR,GRADE
0,NCO-01-01,ΜΑΘΗΜΑΤΙΚΗ ΑΝΑΛΥΣΗ Ι,Υ,2017 - 2018,ΦΕΒΡ,5.0,4.0,5.0,10
1,NCO-01-02,ΕΙΣΑΓΩΓΗ ΣΤΗΝ ΠΛΗΡΟΦΟΡΙΚΗ,Υ,2017 - 2018,ΦΕΒΡ,4.0,3.0,4.0,10
2,NCO-01-03,ΓΡΑΜΜΙΚΗ ΑΛΓΕΒΡΑ,Υ,2017 - 2018,ΦΕΒΡ,5.0,4.0,5.0,10
3,NCO-01-04,ΔΙΑΚΡΙΤΑ ΜΑΘΗΜΑΤΙΚΑ,Υ,2017 - 2018,ΦΕΒΡ,7.0,5.0,7.0,9
4,NCO-01-05,ΒΑΣΙΚΕΣ ΑΡΧΕΣ ΠΡΟΓΡΑΜΜΑΤΙΣΜΟΥ,Υ,2017 - 2018,ΦΕΒΡ,9.0,7.0,9.0,10
5,NCO-02-01,ΜΑΘΗΜΑΤΙΚΗ ΑΝΑΛΥΣΗ ΙΙ,Υ,2017 - 2018,ΙΟΥΝ,5.0,4.0,5.0,8
6,NCO-02-02,ΠΙΘΑΝΟΤΗΤΕΣ & ΣΤΑΤΙΣΤΙΚΗ,Υ,2017 - 2018,ΙΟΥΝ,5.0,4.0,5.0,8
7,NCO-02-03,ΔΟΜΕΣ ΔΕΔΟΜΕΝΩΝ,Υ,2017 - 2018,ΙΟΥΝ,7.0,6.0,7.0,10
8,NCO-02-04,ΨΗΦΙΑΚΗ ΣΧΕΔΙΑΣΗ,Υ,2017 - 2018,ΙΟΥΝ,7.0,6.0,7.0,9
9,NCO-02-05,ΘΕΩΡΙΑ ΥΠΟΛΟΓΙΣΜΟΥ,Υ,2017 - 2018,ΙΟΥΝ,6.0,5.0,6.0,10


In [10]:
# Find more information on the courses
unique_courses = csd_grades["C_ID"].unique()
print("Total unique courses:{}".format(len(unique_courses)))
csd_grades["C_ID"].value_counts()

Total unique courses:101


NCO-02-05     37
NCO-02-03     37
NCO-02-01     37
NCO-01-01     37
NCO-02-02     36
NCO-01-02     36
NCO-01-05     35
NCO-01-04     35
NCO-01-03     34
NCO-02-04     33
NCO-03-01     32
NCO-04-05     32
NCO-03-05     32
NCO-03-02     32
NCO-04-02     32
NCO-04-04     32
NCO-04-03     31
NCO-04-01     31
NCO-03-04     31
NCO-03-03     29
NCO-05-03     13
NCO-05-04     12
NCO-05-02     12
NCO-05-05     12
NCO-05-01     11
NNA-06-04      9
NIS-06-01      6
NIS-08-02      6
NIS-06-04      6
NGE-06-02      6
              ..
NGE-07-02      2
NNA-07-06      2
NET-07-02      2
NIS-07-02      2
CO-01-01       2
NET-07-03      1
CO-04-04       1
NDM-08-0 2     1
ER-01-04       1
NDM-06-0 4     1
CO-04-02       1
NGE-08-02      1
ER-01-03       1
CO-03-03       1
CO-04-03       1
NDM-07-0 2     1
NDM-07-0 1     1
CO-03-07       1
CO-04-07       1
NET-08-01      1
NNA-07-02      1
NDM-07-0 5     1
NDM-06-0 3     1
NIS-08-04      1
CO-04-06       1
NIS-07-08      1
NDM-08-0 3     1
NIS-07-06     

In [11]:
import numpy as np
colnames = ["C_ID","C_NAME","TYPE","EXAM_YEAR","EXAM_PERIOD","ECTS","DM","FACTOR","GRADE"]
course_dictionary = dict()
# Create a dictionary for each course, each row represents a student
for course in unique_courses:
    course_dictionary[course.replace(" ","")] = [-1] * len(csd_csv_files) 
for i in range(0,len(csd_csv_files)):
    csv_file = csd_csv_files[i]
    f = open(csv_file,encoding='utf-8')
    df = pd.read_csv(f, index_col=None, header=None, names = colnames)
    for course_id in df["C_ID"].to_numpy():
        course_dictionary[course_id.replace(" ","")][i] =  df[df["C_ID"] == course_id]["GRADE"].to_numpy()[0]
csd_courses_df = pd.DataFrame.from_dict(course_dictionary)
display(csd_courses_df)

,NCO-01-01,NCO-01-02,NCO-01-03,NCO-01-04,NCO-01-05,NCO-02-01,NCO-02-02,NCO-02-03,NCO-02-04,NCO-02-05,...,NIS-07-08,NDM-07-01,NDM-07-02,NDM-08-02,NGE-08-02,NIS-07-06,NDM-06-03,NDM-07-05,NDM-08-03,NIS-08-04
0,10,10,10,9,10,8,8,10,9,10,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
1,6,10,7,8,7,8,5,7,9,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
2,6,7,5,10,9,6,5,6,7,5,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
3,7,10,8,9,10,6,8,8,9,9,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
4,10,10,10,10,10,10,8,10,9,10,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
5,-1,-1,-1,-1,-1,-1,-1,8,-1,6,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
6,6,9,9,7,9,7,8,7,1,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
7,8,10,10,9,10,10,9,9,10,10,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
8,4,9,4,6,10,7,,7,9,,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
9,-1,-1,-1,-1,-1,-1,-1,9,-1,-1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1


In [12]:
# Fill nan values with -1
csd_courses_df.fillna(-1,inplace = True)
# Replace blank values with -1
csd_courses_df.replace(r'^\s*$', -1, regex=True,inplace=True)
# Replace ΕΠΙΤ with 10
csd_courses_df.replace(' ΕΠΙΤ',10,inplace=True)
display(csd_courses_df)

,NCO-01-01,NCO-01-02,NCO-01-03,NCO-01-04,NCO-01-05,NCO-02-01,NCO-02-02,NCO-02-03,NCO-02-04,NCO-02-05,...,NIS-07-08,NDM-07-01,NDM-07-02,NDM-08-02,NGE-08-02,NIS-07-06,NDM-06-03,NDM-07-05,NDM-08-03,NIS-08-04
0,10,10,10,9,10,8,8,10,9,10,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
1,6,10,7,8,7,8,5,7,9,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
2,6,7,5,10,9,6,5,6,7,5,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
3,7,10,8,9,10,6,8,8,9,9,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
4,10,10,10,10,10,10,8,10,9,10,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
5,-1,-1,-1,-1,-1,-1,-1,8,-1,6,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
6,6,9,9,7,9,7,8,7,1,7,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
7,8,10,10,9,10,10,9,9,10,10,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
8,4,9,4,6,10,7,-1,7,9,-1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1
9,-1,-1,-1,-1,-1,-1,-1,9,-1,-1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1,-1,-1


In [13]:
# Change to type flaot
csd_courses_df = csd_courses_df.astype(np.float64)

In [14]:
csd_courses_df.describe()

,NCO-01-01,NCO-01-02,NCO-01-03,NCO-01-04,NCO-01-05,NCO-02-01,NCO-02-02,NCO-02-03,NCO-02-04,NCO-02-05,...,NIS-07-08,NDM-07-01,NDM-07-02,NDM-08-02,NGE-08-02,NIS-07-06,NDM-06-03,NDM-07-05,NDM-08-03,NIS-08-04
count,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000,...,39.000000,39.0,39.000000,39.0,39.000000,39.000000,39.000000,39.000000,39.000000,39.000000
mean,5.846154,7.512821,5.794872,5.307692,5.897436,5.846154,6.089744,6.102564,5.653846,5.179487,...,-0.871795,-1.0,-0.794872,-1.0,-0.794872,-0.717949,-0.794872,-0.743590,-0.794872,-0.794872
std,3.232552,3.315607,3.735882,3.525500,4.266114,3.289045,3.069099,3.522819,4.001897,3.307661,...,0.800641,0.0,1.281025,0.0,1.281025,1.761410,1.281025,1.601282,1.281025,1.281025
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,5.000000,7.000000,4.000000,4.000000,2.000000,5.000000,5.000000,6.000000,2.500000,4.000000,...,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,6.000000,9.000000,7.000000,6.000000,7.000000,6.000000,7.000000,7.000000,7.000000,5.000000,...,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
75%,8.000000,10.000000,8.000000,8.000000,9.500000,8.000000,8.000000,8.000000,9.000000,7.000000,...,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,4.000000,-1.0,7.000000,-1.0,7.000000,10.000000,7.000000,9.000000,7.000000,7.000000


In [15]:
# Plot some histograms and check distributions
csd_courses_df.hist(layout=(34,3),figsize=(50,500),sharex=True,sharey=True,bins=11)
plt.show()

#### Concatenate CSD Student + Grades ####

In [16]:
# Remove unnecessary fields for final dataset
csd_final = csd.drop(['Timestamp', 'Σχολή Φοίτησης',
       'Υποβολή αναλυτικής βαθμολογίας',
       'Συναινώ στη χρήση των στοιχείων μου από την ομάδα του Grade++',
       'Αν θες να λάβεις πρώτος πρόσβαση στην εφαρμογή, συμπλήρωσε το email σου:',
       'Filename', 'CSV'],axis=1)
csd_final.reset_index(drop=True,inplace=True)
display(csd_final)

,Ηλικία,Φύλο,Επέλεξα τη σχολή μου διότι:,"Κατά μέσο όρο την εβδομάδα, διαβάζω:","Μέσα στο εξάμηνο, παρακαλουθώ:",Υπήρξε ανάγκη για φροντηστηριακή βοήθεια σε κάποιο μάθημα έως τώρα;,"Μετά το πτυχίο, θα ήθελα να ακολουθήσω:",Ποιο από τα παρακάτω ισχύει;,Η σχολή απέχει από το σπίτι μου:,Ασχολούμαι εβδομαδιαία με:
0,19,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,Σειρές / Ταινίες
1,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες"
2,22,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,2 - 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,"Αθλητισμό, Σειρές / Ταινίες, Εθελοντισμός"
3,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες"
4,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Εθελοντισμός"
5,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,2 - 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,25 - 45 λεπτά,"Σειρές / Ταινίες, Εθελοντισμός"
6,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Συγκατοικώ με τα αδέρφια μου,10 - 25 λεπτά,"Αθλητισμό, Video Games"
7,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,< 10 λεπτά,"Video Games, Ξένη γλώσσα, Σειρές / Ταινίες"
8,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Σειρές / Ταινίες, Πανεπιστημιακές..."
9,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,10 - 25 λεπτά,"Αθλητισμό, Σειρές / Ταινίες"


In [17]:
# Concat with grades
ldf = [csd_final, csd_courses_df]
csd_final = pd.concat(ldf,axis=1)
display(csd_final)

,Ηλικία,Φύλο,Επέλεξα τη σχολή μου διότι:,"Κατά μέσο όρο την εβδομάδα, διαβάζω:","Μέσα στο εξάμηνο, παρακαλουθώ:",Υπήρξε ανάγκη για φροντηστηριακή βοήθεια σε κάποιο μάθημα έως τώρα;,"Μετά το πτυχίο, θα ήθελα να ακολουθήσω:",Ποιο από τα παρακάτω ισχύει;,Η σχολή απέχει από το σπίτι μου:,Ασχολούμαι εβδομαδιαία με:,...,NIS-07-08,NDM-07-01,NDM-07-02,NDM-08-02,NGE-08-02,NIS-07-06,NDM-06-03,NDM-07-05,NDM-08-03,NIS-08-04
0,19,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,Σειρές / Ταινίες,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,22,Κορίτσι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,2 - 5 ώρες,Παραπάνω από τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,25 - 45 λεπτά,"Αθλητισμό, Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Αθλητισμό, Video Games, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,2 - 5 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Διδακτορικές Σπουδές,Μένω μόνος/-η μου,25 - 45 λεπτά,"Σειρές / Ταινίες, Εθελοντισμός",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
6,19,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Περίπου τις μισές διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Συγκατοικώ με τα αδέρφια μου,10 - 25 λεπτά,"Αθλητισμό, Video Games",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,0 - 2 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,< 10 λεπτά,"Video Games, Ξένη γλώσσα, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8,20,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο...,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Μένω μόνος/-η μου,< 10 λεπτά,"Video Games, Σειρές / Ταινίες, Πανεπιστημιακές...",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9,22,Αγόρι,Μου αρέσει η ενασχόληση με αυτό το αντικείμενο.,> 5 ώρες,Όλες τις διαλέξεις,Όχι,Μεταπτυχιακές Σπούδες,Ζω με τους γονείς μου (ή γενικότερα την οικογέ...,10 - 25 λεπτά,"Αθλητισμό, Σειρές / Ταινίες",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [18]:
# Write to file
csd_final.to_csv("csd.csv",index=False)